# Web Spider for AFL Data
#### Quick Start

In [1]:
# import libraries
import requests
from fake_useragent import UserAgent
from time import sleep

from bs4 import BeautifulSoup
import re

In [2]:
# Crawling Website
url='https://afltables.com/afl/seas/2022.html'
# Fake a user header
headers = {'User-Agent': UserAgent().random}


datas = requests.get(url=url, headers=headers)
html = datas.text

# For Loop crawling, time interval is demanded.
sleep(1)

In [3]:
soup = BeautifulSoup(html, 'lxml')


In [17]:
games = soup.find_all('table', attrs={'border':1})

for game in games:
    tables = game.find_all('td')
    for table in tables:
        print(table)
        print('#########')
    print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')


<td width="16%"><a href="../teams/melbourne_idx.html">Melbourne</a></td>
#########
<td align="center" nowrap="" width="20%"><tt>  4.5   6.6  10.8 14.13 </tt></td>
#########
<td align="center" width="5%"> 97</td>
#########
<td>Wed 16-Mar-2022 7:10 PM <b>Att: </b>58,002 <b>Venue:</b> <a href="../venues/mcg.html">M.C.G.</a></td>
#########
<td width="16%"><a href="../teams/bullldogs_idx.html">Western Bulldogs</a></td>
#########
<td align="center" nowrap="" width="20%"><tt>  2.2   8.3   8.5  11.5 </tt></td>
#########
<td align="center" width="5%"> 71</td>
#########
<td><b>Melbourne</b> won by <b>26 pts </b>[<a href="../stats/games/2022/071120220316.html">Match stats</a>]</td>
#########
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
<td width="16%"><a href="../teams/carlton_idx.html">Carlton</a></td>
#########
<td align="center" nowrap="" width="20%"><tt>  1.3   5.9  7.12 14.17 </tt></td>
#########
<td align="center" width="5%"> 101</td>
#########
<td>Thu 17-Mar-2022 7:25 PM <b>Att: </b>72,179 <b>

In [5]:
# Melbourne
#contentpagecell > table > tbody > tr > td:nth-child(1) > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(3) > td:nth-child(3) > table > tbody > tr:nth-child(2) > td.ylwbg > table > tbody > tr:nth-child(1) > td:nth-child(1) > a

In [6]:
# 97
#contentpagecell > table > tbody > tr > td:nth-child(1) > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(3) > td:nth-child(3) > table > tbody > tr:nth-child(2) > td.ylwbg > table > tbody > tr:nth-child(1) > td:nth-child(2) > b

In [7]:
# Western Bulldogs
#contentpagecell > table > tbody > tr > td:nth-child(1) > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(3) > td:nth-child(3) > table > tbody > tr:nth-child(2) > td.ylwbg > table > tbody > tr:nth-child(1) > td:nth-child(4) > a

In [8]:
# 71
#contentpagecell > table > tbody > tr > td:nth-child(1) > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(3) > td:nth-child(3) > table > tbody > tr:nth-child(2) > td.ylwbg > table > tbody > tr:nth-child(1) > td:nth-child(5) > b

In [9]:
# MCG
#contentpagecell > table > tbody > tr > td:nth-child(1) > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(3) > td:nth-child(3) > table > tbody > tr:nth-child(2) > td.ylwbg > table > tbody > tr:nth-child(1) > td:nth-child(7)

In [10]:
# Date
#contentpagecell > table > tbody > tr > td:nth-child(1) > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(3) > td:nth-child(3) > table > tbody > tr:nth-child(2) > td.ylwbg > table > tbody > tr:nth-child(3) > td

In [11]:
# stats
#contentpagecell > table > tbody > tr > td:nth-child(1) > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(3) > td:nth-child(3) > table > tbody > tr:nth-child(2) > td.ylwbg > table > tbody > tr:nth-child(3) > td > a:nth-child(1)

In [12]:
# Carlton
#contentpagecell > table > tbody > tr > td:nth-child(1) > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(5) > td:nth-child(3) > table > tbody > tr:nth-child(2) > td.ylwbg > table > tbody > tr:nth-child(1) > td:nth-child(1) > a

In [13]:
# St Kiilda
#contentpagecell > table > tbody > tr > td:nth-child(1) > table > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(7) > td:nth-child(3) > table > tbody > tr:nth-child(2) > td.ylwbg > table > tbody > tr:nth-child(1) > td:nth-child(1) > a